# mnist 학습

In [1]:
import tensorflow
from tensorflow.keras.datasets import mnist
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data() # MNIST 에 있는 훈련 세트와 테스트 세트를 가져와 저장


train_images = train_images.reshape((60000, 28 * 28))
# 60000, 784
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

- 입력 이미지의 데이터 타입
    - float32
- 훈련 데이터는 (60000, 784) 테스트 데이터는 (10000, 784) 크기의 넘파이 배열로 저장

In [3]:
model = keras.Sequential([
    layers.Dense(512, activation='relu'),
    layers.Dense(10, activation='softmax')
])

- 2개의 Dense(fully connected) 층이 연결되어 있음
- 각 층은 가중치 텐서를 포함하여 입력 데이터에 대한 몇 개의 간단한 텐서 연산을 적용
    - 가중치 텐서는 모델이 정보를 저장하는 곳

In [4]:
model.compile(optimizer="rmsprop", 
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

- sparse_categoical_crossentropy
    - 손실 함수
    - 가중치 텐서를 학습하기 위한 피드백 신호로 사용
    - 훈련하는 동안 최소화됨
- rmsprop
    - 미니 배치 확률적 경사 하강법을 통해 손실이 감소

[Optimizer 종류 및 정리](https://velog.io/@chang0517/Optimizer-종류-및-정리)

- accuracy
    - 훈련 척도를 판단

In [25]:
type(model)

keras.engine.sequential.Sequential

In [ ]:
model.compil(optimizer=keras.optimizers.RMSprop(learning_rate=1e-4),
             loss=keras.losses.MeanSquaredError(),
             metrics=[keras.metrics.BinaryAccuracy])

In [5]:
model.fit(train_images, train_labels, epochs=5, batch_size=128)
# 모델을 train_images와 train_labels를 이용하여 훈련시킴.

Epoch 1/5
469/469 [==============================] - 4s 8ms/step - loss: 0.2605 - accuracy: 0.9252
Epoch 2/5
469/469 [==============================] - 4s 9ms/step - loss: 0.1038 - accuracy: 0.9694
Epoch 3/5
469/469 [==============================] - 4s 9ms/step - loss: 0.0677 - accuracy: 0.9799
Epoch 4/5
469/469 [==============================] - 4s 9ms/step - loss: 0.0490 - accuracy: 0.9859
Epoch 5/5
469/469 [==============================] - 4s 9ms/step - loss: 0.0370 - accuracy: 0.9888


- fit() method 호출
    - 모델이 128개 샘플ㄹ의 미니 배치로 훈련 데이터를 다섯 번 반복
        - 전체 훈련 데이터에 수행되는 각 반복을 에포크(epoch)라고 함
    - 각 배치에서 모델이 가중치에 대한 손실의 그래디언트를 계산
        - 이 배치에서 손실 값을 감소시키는 방향으로 가중치를 이동

In [6]:
test_digits = test_images[0:10]
predictions = model.predict(test_digits)
predictions[0]

1/1 [==============================] - 0s 127ms/step


array([8.1761653e-10, 1.2177274e-12, 1.0572943e-06, 4.5933384e-06,
       7.3575868e-13, 3.4756656e-08, 4.7727245e-15, 9.9999386e-01,
       6.9427223e-08, 2.3376178e-07], dtype=float32)

In [7]:
predictions[0].argmax()
# 가장 큰 값을 가지는 인덱스를 출력

7

In [8]:
predictions[0][7]
# predictions[0]이 7일 확률

0.99999386

In [9]:
test_labels[0]
# test_labels의 값을 확인
# 7이므로 예측에 성공했음

7

In [10]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"테스트 정확도: {test_acc}")

313/313 [==============================] - 1s 2ms/step - loss: 0.0665 - accuracy: 0.9800
테스트 정확도: 0.9800000190734863


# 원소별 연산

In [11]:
import numpy as np
import time

In [12]:
x = np.random.random((20, 100))
y = np.random.random((20, 100))

### relu 함수 구현

In [13]:
def naive_relu(x) :
    assert len(x.shape) == 2 # x는 랭크-2 넘파이 배열
    x = x.copy()  # 입력 텐서 자체를 바꾸지 않도록 복사
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] = max(x[i, j], 0)
    return x

### 덧셈

In [14]:
def naive_add(x, y):
    assert len(x.shape) == 2 # x는 랭크-2 넘파이 배열
    assert x.shape == y.shape
    x = x.copy()  # 입력 텐서 자체를 바꾸지 않도록 복사
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[i, j]
    return x

# 넘파이 원소별 연산

In [15]:
t0 = time.time()
z = naive_add(x, y)
r = naive_relu(z)

print(r)
print("걸린 시간: {0: 2f} s".format(time.time() - t0))


[[0.21955958 0.7019504  0.34630893 ... 0.90744279 0.93075994 1.03072054]
 [1.20842744 1.33730377 0.33587193 ... 1.74542702 0.81193457 0.95162382]
 [1.66051933 0.809951   1.25767017 ... 0.2126789  0.77049905 1.1472142 ]
 ...
 [1.39456061 0.17034732 0.64846586 ... 0.48722362 0.69081215 0.25014808]
 [0.72451213 1.05000398 0.54256297 ... 1.10855373 0.76943757 1.84872601]
 [0.77843913 0.56484923 1.17622587 ... 0.37550032 1.05307779 1.09232998]]
걸린 시간:  0.008975 s


In [16]:
t0 = time.time()
z = x + y # 원소별 덧셈
z = np.maximum(z, 0.) # 원소별 렐루 함수
print(r)
print("걸린 시간: {0: 2f} s".format(time.time() - t0))

[[0.21955958 0.7019504  0.34630893 ... 0.90744279 0.93075994 1.03072054]
 [1.20842744 1.33730377 0.33587193 ... 1.74542702 0.81193457 0.95162382]
 [1.66051933 0.809951   1.25767017 ... 0.2126789  0.77049905 1.1472142 ]
 ...
 [1.39456061 0.17034732 0.64846586 ... 0.48722362 0.69081215 0.25014808]
 [0.72451213 1.05000398 0.54256297 ... 1.10855373 0.76943757 1.84872601]
 [0.77843913 0.56484923 1.17622587 ... 0.37550032 1.05307779 1.09232998]]
걸린 시간:  0.000991 s


# 브로드캐스팅

In [17]:
x = np.random.random((300, 200))
y = np.random.random((200))
x.shape[0]

300

In [18]:
# y에 비어 있는 첫 번째 축을 추가하여 크기를 (1, 10)으로 만든다
y = np.expand_dims(y, axis=0)

y.shape

(1, 200)

In [19]:
# y를 이 축에 32번 반복
# 텐서 y의 크기는 (32, 10)이 된다.
# y[i, :] == y for i in range(0, 32)
# 실제로 이루어지면 비요율적이므로 알고리즘 수준에서 일어남

y = np.concatenate([y] * 300, axis=0)
y.shape

(300, 200)

In [20]:
def naive_add_matrix_and_vector(x, y) :
    assert len(x.shape) == 2 # 행렬
    assert len(y.shape) == 1 # 벡터
    assert x.shape[1] == y.shape[0]
    x = x.copy() # 입력 텐서 자체를 바꾸지 않도록 복사
    for i in range(x.shape[0]) : # len(x)
        for j in range(x.shape[1]) : # len(x[])
            x[i, j] += y[j]
    return x

In [21]:
t0 = time.time()
z = x + y

print("걸린 시간: {0: 2f} s".format(time.time() - t0))
z.shape

걸린 시간:  0.000000 s


(300, 200)

In [22]:
t0 = time.time()
z = naive_add_matrix_and_vector(x, y)

print("걸린 시간: {0: 2f} s".format(time.time() - t0))
z.shape

AssertionError: 

# 텐서 곱셈(점곱)

In [ ]:
x = np.random.random((300, 100))
y = np.random.random((100, 200))
n = np.dot(x, y)

print(n.shape)


(300, 200)


In [ ]:
def my_matrix_dot(x, y) :
    assert len(x.shape) == len(y.shape)
    assert x.shape[1] == y.shape[0]
    z = np.zeros((x.shape[0], y.shape[1]), dtype=float)
    for i in range(x.shape[0]) :
        for j in range(y.shape[1]) :
            for k in range(x.shape[1]) :
                z[i, j] += x[i, k] * y[k, j]
    return z

In [ ]:
z = np.dot(x, y)

[[0.21220707 0.32122658 0.30471821 ... 0.16879869 0.71069028 0.43218136]
 [0.94995394 0.27114344 0.31901023 ... 0.07357104 0.62965391 0.35025315]
 [0.44073104 0.88724855 0.86742679 ... 0.19789265 0.43693749 0.28512642]
 ...
 [0.3841409  0.7883325  0.96652127 ... 0.08439272 0.91571374 0.88751818]
 [0.5766854  0.19581061 0.42890274 ... 0.79426132 0.22968651 0.28816944]
 [0.22826123 0.12512395 0.83863153 ... 0.46675371 0.41773987 0.5221204 ]] [[0.60363849 0.87238091 0.77833025 ... 0.49751104 0.42692533 0.81706044]
 [0.76577703 0.05973845 0.98206379 ... 0.63923827 0.280827   0.54323107]
 [0.61449565 0.36020628 0.6221354  ... 0.35607232 0.22010692 0.98999782]
 ...
 [0.14703719 0.19597317 0.66806201 ... 0.67315202 0.58206843 0.42091061]
 [0.94653852 0.65163446 0.13741731 ... 0.89770702 0.94822923 0.95783474]
 [0.75324721 0.64045216 0.13173775 ... 0.28758528 0.15215775 0.8341417 ]]
[[25.43261795 24.35578357 24.82861103 ... 26.90587443 27.10773216
  25.1365987 ]
 [26.09851501 25.56460397 27.14

In [ ]:
z = my_matrix_dot(x, y)
print(z)

[[25.43261795 24.35578357 24.82861103 ... 26.90587443 27.10773216
  25.1365987 ]
 [26.09851501 25.56460397 27.14648228 ... 27.05514095 24.32427744
  23.99167037]
 [27.89192095 26.57123732 26.0129996  ... 27.03318337 26.36453993
  26.2000945 ]
 ...
 [26.75828064 24.04978039 26.21163212 ... 25.84443693 23.58633236
  25.49478373]
 [25.59086458 25.42814512 26.80227265 ... 26.45521765 23.64845261
  24.59164668]
 [24.77692272 25.2107622  25.76937185 ... 28.0395703  24.46572979
  24.45086963]]


In [ ]:
def naive_vector_dot(x, y) :
    assert len(x.shape) == 1
    assert len(y.shape) == 1
    assert x.shape[0] == y.shape[0]
    z = 0.
    for i in range(x.shape[0]) :
        z += x[i] * y[i]
    return z

In [ ]:
def naive_matrix_dot(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 2
    assert x.shape[1] == y.shape[0]
    z = np.zeros((x.shape[0], y.shape[1]))
    for i in range(x.shape[0]):
        for j in range(y.shape[1]):
            row_x = x[i, :]
            column_y = y[:, j]
            z[i, j] = naive_vector_dot(row_x, column_y)
    return z

In [ ]:
z = naive_matrix_dot(x, y)
print(z)

[[25.43261795 24.35578357 24.82861103 ... 26.90587443 27.10773216
  25.1365987 ]
 [26.09851501 25.56460397 27.14648228 ... 27.05514095 24.32427744
  23.99167037]
 [27.89192095 26.57123732 26.0129996  ... 27.03318337 26.36453993
  26.2000945 ]
 ...
 [26.75828064 24.04978039 26.21163212 ... 25.84443693 23.58633236
  25.49478373]
 [25.59086458 25.42814512 26.80227265 ... 26.45521765 23.64845261
  24.59164668]
 [24.77692272 25.2107622  25.76937185 ... 28.0395703  24.46572979
  24.45086963]]


# 텐서 크기 변환

In [ ]:
x = np.array([[0., 1.],
             [2., 3.],
             [4., 5.]])

In [ ]:
x = x.reshape((6, 1))
x

array([[0.],
       [1.],
       [2.],
       [3.],
       [4.],
       [5.]])

In [ ]:
x = x.reshape(1, 6)
x

array([[0., 1., 2., 3., 4., 5.]])

# 연쇄 법칙

In [ ]:
def fg(x) :
    x1 = g(x)
    y = f(x1)
    return y

In [ ]:
grad(y, x) == grad(x1, x)

In [ ]:
def fghj(x) :
    x1 =j(x)
    x2 = h(x1)
    x3 = g(x2)
    y = f(x3)
    return y

In [ ]:
grad(y, x) == (grad(y, x3) * grad(x3, x2) * grad(x2, x1), grad(x1, x))